# I- DESCRIPTION

In [ ]:
qMRinfo('qmt_sirfse'); % Describe the model

# II- MODEL PARAMETERS
## a- Create object

In [ ]:
Model = qmt_sirfse;

## Download sample data from OSF
> The current `Model` is an instance of `qmt_sirfse` class.
 
You can manually download the sample data for `qmt_sirfse` [by clicking here](https://osf.io/fk2nd/download?version=2).

In [ ]:
dataDir = downloadData(Model,pwd);

## b- Set protocol
> Protocol is set according to the example data
 
%           |- qmt_sirfse object needs 2 protocol field(s) to be assigned:
%           |-   MTdata
%           |-   FSEsequence

In [ ]:
% Ti is a vector of [25X1]
Ti = [0.0030; 0.0037; 0.0047; 0.0058; 0.0072; 0.0090; 0.0112; 0.0139; 0.0173; 0.0216; 0.0269; 0.0335; 0.0417; 0.0519; 0.0646; 0.0805; 0.1002; 0.1248; 0.1554; 0.1935; 0.2409; 0.3000; 1.0000; 2.0000; 10.0000];
% Td is a vector of [25X1]
Td = [3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000; 3.5000];
Model.Prot.MTdata.Mat = [ Ti Td];
% -----------------------------------------
Trf  = 0.001;
Tr  = 0.01;
Npulse = 16;
Model.Prot.FSEsequence.Mat = [ Trf  Tr  Npulse];
% -----------------------------------------

# III- FIT EXPERIMENTAL DATASET
## a- Load experimental data
%           |- qmt_sirfse object needs 3 data input(s) to be assigned:
%           |-   MTdata
%           |-   R1map
%           |-   Mask

In [ ]:
% MTdata.nii.gz contains [128  128    1   25] data.
data.MTdata=double(load_nii_data('qmt_sirfse_data/MTdata.nii.gz'));
% Mask.nii.gz contains [128  128] data.
data.Mask=double(load_nii_data('qmt_sirfse_data/Mask.nii.gz'));
 

## b- Fit experimental data
> This section will fit data.

In [ ]:
FitResults = FitData(data,Model,0);

## c- Show fitting results
> * Output map will be displayed.
 
> * If available, a graph will be displayed to show fitting in a voxel.

In [ ]:
qMRshowOutput(FitResults,data,Model);

## d- Save results
> * qMR maps are saved in NIFTI and in a structure `FitResults.mat` that can be loaded in qMRLab graphical user interface.
> * Model object stores all the options and protocol
> * These objects can be easily shared or be used for simulation.

In [ ]:
FitResultsSave_nii(FitResults, 'qmt_sirfse_data/MTdata.nii.gz');

# IV- SIMULATIONS
> This section can be executed to run simulations for vfa_t1.
 
## a- Single Voxel Curve
> Simulates single voxel curves:
 
       1. Use equation to generate synthetic MRI data
       2. Add rician noise
       3. Fit and plot curve

In [ ]:
      x = struct;
      x.F = 0.1;
      x.kr = 30;
      x.R1f = 1;
      x.R1r = 1;
      x.Sf = -0.98419;
      x.Sr = 0.65638;
      x.M0f = 1;
      % Set simulation options
      Opt.SNR = 50;
      Opt.Method = 'Analytical equation';
      Opt.T2fUsedinBlockequation = 0.04;
      % run simulation
      figure('Name','Single Voxel Curve Simulation');
      FitResult = Model.Sim_Single_Voxel_Curve(x,Opt);

## b- Sensitivity analysis
> Simulates sensitivity to fitted parameters: 
 
       1. Vary fitting parameters from lower (lb) to upper (ub) bound.
       2. Run Sim_Single_Voxel_Curve Nofruns times
       3. Compute mean and std across runs

In [ ]:
      %              F             kr            R1f           R1r           Sf            Sr            M0f           
      OptTable.st = [0.1           30            1             1             -0.98         0.66          1]; % nominal values
      OptTable.fx = [0             1             1             1             1             1             1]; %vary F...
      OptTable.lb = [0.0001        0.0001        0.05          0.05          -1            0.0001        0.0001]; %...from 0.0001
      OptTable.ub = [1             1e+02         10            10            0             1             2]; %...to 1
      % Set simulation options
      Opt.SNR = 50;
      Opt.Method = 'Analytical equation';
      Opt.T2fUsedinBlockequation = 0.04;
      Opt.Nofrun = 5;
      % run simulation
      SimResults = Model.Sim_Sensitivity_Analysis(OptTable,Opt);
      figure('Name','Sensitivity Analysis');
      SimVaryPlot(SimResults, 'F' ,'F' );